# Inspect source light curves and skymaps

### Setup main notebook options

In [1]:
region_name = "ALL_10-800_LOOSE"  #"TDS" #"CAINGSGII_10-800"  #"TDS" #"WD" #"MDIS_10-800" # "TDS_ELAISN1"
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"
show_skymaps = False
show_sed = False
load_src_from_file = False
lc_block_srcs = 5 #Setup light curve plotting parameters

#srcs_ids = [2996 ,11821 ,55034 ,61491 ,64360 ,73354 ,76243 ,92867 ,104078 ,106745 ,124860 ,162490 ,182622 ,199884 ,226975 ,267514 ,588567 ,599887 ,789511 ,814134 ,] # WD and candidates, including GAIA info
srcs_ids = []
#srcs_ids = [8215 ,32448 ,54184 ,98746 ,151796 ,166179 ,172775 ,193067 ,199832 ,259271 ,265150 ,297278 ,305192 ,388172 ,432606 ,472623 ,535864 ,541266 ,625693 ,1521738 ,1551422 ,2136829 ,] # WD ALL_10-800-LOOSE
srcs_ids = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,
            253670 ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,
            645215 ,645730 ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206
            ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE
srcs_ids = [309 ,332 ,473 ,855 ,2002 ,4239 ,4284 ,4530 ,4780 ,4943 ,5699 ,7437 ,7828 ,8322 ,8775 ,9455 ,9547 ,9697 ,10624 ,11094 ,11250 ,11995 ,12019 ,12891 ,13691 ,14219 ,15357 ,16743 ,19905 ,20345 ,20831 ,21689 ,21974 ,22078 ,22339 ,23708 ,24733 ,24792 ,25087 ,25424 ,25621 ,25756 ,25820 ,25994 ,29741 ,30079 ,30628 ,30742 ,31844 ,31895 ,32121 ,32240 ,32792 ,32865 ,33047 ,33138 ,33145 ,33699 ,33729 ,33928 ,34736 ,35107 ,35343 ,35689 ,40510 ,42089 ,46187 ,46339 ,46367 ,46478 ,46779 ,46900 ,47453 ,47910 ,47921 ,48106 ,48249 ,48531 ,48609 ,48610 ,48908 ,49738 ,51336 ,51884 ,51923 ,52997 ,53020 ,53128 ,53240 ,53347 ,54471 ,55355 ,55640 ,56136 ,56173 ,56335 ,56339 ,57032 ,57209 ,57272 ,57299 ,57311 ,57433 ,57981 ,57997 ,58019 ,59891 ,59898 ,60479 ,60636 ,60728 ,61033 ,61106 ,62239 ,62766 ,62894 ,63280 ,63400 ,63769 ,63988 ,64135 ,64323 ,64826 ,65271 ,65422 ,65806 ,66342 ,66636 ,67397 ,67491 ,67564 ,67592 ,67621 ,67792 ,67854 ,68291 ,69320 ,69585 ,69972 ,70435 ,73488 ,75681 ,79102 ,79814 ,80655 ,83200 ,83329 ,84538 ,85251 ,85662 ,86163 ,87171 ,87205 ,89840 ,89967 ,91175 ,91416 ,92874 ,94537 ,100686 ,100974 ,109430 ,111471 ,121016 ,121028 ,121939 ,121998 ,122430 ,122762 ,124621 ,125042 ,125235 ,125264 ,125419 ,126134 ,126729 ,127025 ,131049 ,134146 ,137229 ,142787 ,143332 ,144766 ,144771 ,144787 ,145382 ,145809 ,146173 ,146479 ,146538 ,146548 ,148422 ,148699 ,149506 ,149589 ,149603 ,149972 ,150316 ,150348 ,150505 ,150722 ,151221 ,151227 ,151235 ,151295 ,152056 ,152390 ,152603 ,152769 ,153141 ,154137 ,155087 ,155769 ,156417 ,157736 ,157775 ,158043 ,158349 ,158667 ,159307 ,159416 ,159417 ,159539 ,159620 ,159648 ,159742 ,159865 ,159889 ,160004 ,160010 ,160132 ,160318 ,161055 ,161930 ,161941 ,162078 ,162184 ,162193 ,162697 ,163516 ,163552 ,164379 ,164889 ,165343 ,165487 ,165988 ,166852 ,166978 ,167607 ,171498 ,171899 ,171917 ,171929 ,174716 ,176901 ,177193 ,177980 ,178161 ,189800 ,191155 ,192306 ,192500 ,192521 ,192576 ,193442 ,193534 ,193683 ,194233 ,194477 ,194576 ,195286 ,196723 ,197070 ,197186 ,198596 ,198637 ,199440 ,199520 ,199653 ,200332 ,200391 ,200487 ,200727 ,200986 ,201340 ,201429 ,202162 ,202482 ,202540 ,203146 ,203287 ,204176 ,204296 ,204448 ,206892 ,207792 ,210220 ,213748 ,214324 ,215227 ,215487 ,215672 ,217805 ,217932 ,217988 ,219426 ,219560 ,219628 ,219820 ,219942 ,221712 ,222369 ,223460 ,226732 ,227709 ,228380 ,231009 ,231076 ,232310 ,233919 ,234568 ,235201 ,236294 ,236438 ,239915 ,240160 ,240277 ,240686 ,240860 ,241754 ,242181 ,253449 ,259191 ,259253 ,259394 ,259472 ,259476 ,259618 ,259677 ,259697 ,259770 ,259775 ,259844 ,259972 ,259995 ,260789 ,260941 ,261051 ,261443 ,261715 ,261790 ,262086 ,262134 ,262774 ,263540 ,264303 ,264919 ,265029 ,265806 ,266127 ,266338 ,266468 ,267970 ,268288 ,268325 ,268501 ,269141 ,269235 ,270179 ,270581 ,271047 ,271532 ,272096 ,272097 ,272978 ,273406 ,273516 ,274314 ,274750 ,274908 ,276672 ,277122 ,277164 ,277349 ,277709 ,278095 ,278242 ,278436 ,278629 ,278886 ,279073 ,279223 ,279797 ,279847 ,280385 ,280547 ,280676 ,281213 ,282177 ,285072 ,285762 ,285887 ,286627 ,286632 ,286676 ,289851 ,294093 ,298020 ,298262 ,298271 ,298449 ,306686 ,307626 ,309363 ,318262 ,320521 ,320746 ,321262 ,321530 ,324026 ,324174 ,324733 ,324891 ,324903 ,324958 ,325135 ,325268 ,325373 ,325538 ,325727 ,326009 ,327444 ,327537 ,327599 ,327606 ,327907 ,328280 ,328945 ,328994 ,329035 ,329067 ,329251 ,329754 ,330698 ,332036 ,332954 ,334526 ,334822 ,334829 ,335384 ,335911 ,336429 ,336754 ,338367 ,339325 ,339514 ,340122 ,340301 ,340464 ,340859 ,340980 ,341391 ,341441 ,341597 ,341624 ,341645 ,341711 ,341735 ,342101 ,342305 ,342678 ,343400 ,344954 ,346276 ,346471 ,346629 ,348404 ,348511 ,351766 ,353218 ,353261 ,353521 ,353889 ,354476 ,355119 ,355234 ,355544 ,355748 ,356549 ,356634 ,385783 ,389222 ,390230 ,390346 ,396417 ,399452 ,400272 ,400401 ,400452 ,400552 ,400965 ,401069 ,401211 ,401425 ,401456 ,401483 ,401656 ,401720 ,401826 ,401877 ,401902 ,402035 ,403226 ,407542 ,407734 ,408036 ,409424 ,409492 ,410556 ,411027 ,416230 ,417460 ,422482 ,422843 ,422962 ,424261 ,425083 ,425579 ,425813 ,426248 ,426726 ,426946 ,427351 ,433417 ,435051 ,439566 ,440257 ,440276 ,441971 ,442065 ,442438 ,442625 ,442908 ,443956 ,444521 ,446697 ,447474 ,449748 ,450210 ,450982 ,451263 ,452429 ,453872 ,454081 ,460044 ,467166 ,469563 ,471716 ,472459 ,474419 ,475321 ,475949 ,477090 ,477556 ,477723 ,480316 ,494514 ,494880 ,495937 ,497369 ,503148 ,503279 ,504045 ,507623 ,510385 ,511809 ,511846 ,512450 ,514944 ,516119 ,519097 ,519381 ,531702 ,531855 ,532106 ,532116 ,534291 ,534322 ,534366 ,534648 ,534735 ,535074 ,535079 ,536702 ,536741 ,536781 ,537841 ,537953 ,538075 ,539638 ,539996 ,541222 ,541915 ,542049 ,542339 ,542663 ,543422 ,543490 ,544632 ,545065 ,545589 ,546424 ,546690 ,546776 ,546817 ,546964 ,546970 ,547028 ,547101 ,547159 ,547193 ,547258 ,547266 ,547298 ,547316 ,548390 ,548456 ,550223 ,550270 ,550941 ,551412 ,552535 ,553213 ,553663 ,554069 ,554107 ,554253 ,554707 ,554859 ,555016 ,555076 ,555524 ,555949 ,556234 ,557215 ,557603 ,558710 ,559797 ,560050 ,560188 ,560465 ,561487 ,561997 ,563640 ,565270 ,565715 ,566780 ,567506 ,567997 ,568915 ,569097 ,574606 ,575361 ,575796 ,576190 ,576271 ,577307 ,578275 ,578342 ,578651 ,578846 ,579038 ,580124 ,581316 ,581492 ,581632 ,581967 ,582268 ,582343 ,582506 ,582618 ,582746 ,583295 ,585162 ,585936 ,591846 ,593012 ,593966 ,594027 ,594100 ,595310 ,596137 ,597356 ,600420 ,600506 ,600964 ,604209 ,609064 ,609596 ,610831 ,612255 ,612699 ,613572 ,613909 ,617607 ,621789 ,624347 ,628637 ,630781 ,631340 ,633429 ,633912 ,639106 ,640193 ,642007 ,642090 ,642153 ,642703 ,644724 ,644828 ,645467 ,645468 ,646035 ,646252 ,646301 ,646461 ,646501 ,647003 ,647269 ,647512 ,647834 ,647963 ,648109 ,648342 ,648383 ,648684 ,648791 ,649020 ,649075 ,649369 ,649449 ,650403 ,650659 ,650847 ,650902 ,657024 ,666974 ,667669 ,672867 ,673098 ,673702 ,673750 ,673989 ,675914 ,676219 ,676364 ,676601 ,677395 ,678663 ,679035 ,682826 ,683128 ,683604 ,683723 ,683980 ,685895 ,686008 ,686459 ,686547 ,690961 ,691014 ,693406 ,694145 ,694401 ,695429 ,696067 ,697705 ,699375 ,699616 ,700005 ,700255 ,700302 ,700665 ,702276 ,702345 ,702996 ,703426 ,703764 ,703881 ,706588 ,707069 ,710450 ,711995 ,713632 ,719409 ,720104 ,720608 ,720659 ,722827 ,722832 ,722984 ,723546 ,726248 ,726405 ,726591 ,728946 ,728978 ,729882 ,731177 ,731678 ,741307 ,741382 ,741658 ,741965 ,742514 ,742630 ,742872 ,742926 ,743695 ,743803 ,743912 ,754068 ,754377 ,755765 ,756091 ,756119 ,756682 ,756892 ,757245 ,759977 ,761439 ,761885 ,772383 ,797079 ,803149 ,803547 ,806790 ,844950 ,845062 ,845631 ,846726 ,905047 ,907750 ,916586 ,945957 ,946176 ,1024458 ,1036492 ,1106654 ,1116253 ,1168576 ,1171118 ,1175164 ,1181606 ,1183312 ,1185970 ,1187478 ,1191082 ,1197830 ,1210196 ,1211440 ,1213574 ,1214552 ,1215576 ,1215825 ,1215962 ,1215966 ,1217085 ,1220555 ,1222361 ,1224898 ,1226216 ,1228231 ,1229666 ,1230673 ,1231540 ,1233030 ,1233524 ,1234157 ,1235949 ,1236466 ,1238108 ,1240877 ,1249571 ,1259156 ,1262794 ,1265495 ,1267194 ,1271961 ,1272368 ,1272786 ,1274449 ,1278193 ,1278222 ,1284883 ,1285162 ,1285917 ,1286107 ,1287296 ,1290004 ,1290925 ,1292334 ,1295169 ,1296463 ,1300090 ,1331418 ,1332086 ,1342181 ,1353446 ,1366015 ,1367233 ,1417054 ,1427467 ,1439734 ,1468570 ,1469385 ,1469826 ,1470569 ,1472471 ,1472650 ,1472932 ,1473677 ,1473792 ,1473972 ,1474604 ,1475179 ,1482789 ,1488012 ,1490421 ,1491988 ,1539429 ,1540818 ,1542523 ,1542934 ,1545822 ,1553083 ,1555740 ,1556253 ,1561498 ,1563035 ,1578603 ,1579732 ,1579840 ,1584522 ,1585411 ,1588073 ,1592262 ,1625785 ,1626212 ,1637503 ,1638586 ,1638627 ,1639464 ,1643727 ,1646374 ,1647025 ,1648811 ,1650401 ,1651290 ,1656150 ,1656576 ,1657842 ,1664778 ,1672849 ,1697519 ,1704092 ,1711016 ,1720618 ,1724416 ,1864702 ,1864995 ,1871162 ,1875404 ,1879257 ,1881920 ,1884477 ,1893810 ,1897630 ,1898282 ,1898995 ,1901515 ,1902191 ,1902362 ,1902426 ,1903809 ,1907966 ,1910140 ,1917225 ,2027968 ,2037328 ,2041853 ,2066342 ,2085716 ,2088315 ,2090376 ,2094903 ,2095358 ,2101075 ,2106440 ,2125483 ,2137338 ,2144507 ,2157677 ,2164936 ,2182895 ,2211541 ,2223332 ,2332808 ,2350716 ,2353172 ,2376462 ,2376905 ,2386614 ,2401354 ,2411632 ,2477456 ,2489797 ,2491604 ,2497909 ,2502578 ,2544519 ,2591747 ,2614723 ,2614791 ,2614874 ,2617045 ,2618719 ,2619822 ,2621096 ,2623979 ,2664799 ,2670740 ,2687309 ,2689183 ,2775976 ,2833417 ,2843738 ,2851986 ,2862717 ,2911876 ,2926770 ,2927320 ,2937357 ,2942355 ,2945045 ,3035825 ,3076627 ,3083503 ,3146735 ,3152250 ,3179747 ,3203759 ,3224680 ,3271696 ,3272327 ,]
srcs_ids = [9697 ,46900 ,48531 ,48609 ,51884 ,57311 ,57433 ,58019 ,63400 ,92874 ,121028 ,121939 ,121998 ,122762 ,144787 ,149972 ,152390 ,156417 ,196723 ,200986 ,204176 ,277122 ,277349 ,277709 ,278242 ,278629 ,279073 ,282177 ,285887 ,286676 ,351766 ,444521 ,537953 ,546424 ,546817 ,547101 ,558710 ,567506 ,646301 ,667620 ,667669 ,673702 ,673750 ,675914 ,679035 ,683128 ,683604 ,683723 ,685895 ,686459 ,696067 ,710450 ,1197830 ,1290004 ,1290925 ,1292334 ,2041853 ,2144507 ,2521386 ,2527903 ,2572986 ,2577984 ,2614791 ,2619822 ,]

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.table import Table
import numpy as np

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, sel_sources, dd_filter2id

from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord

#Load region
rg = Region()
rg.load_from_fits(region_fname)

print("Nr sources:",len(rg.tt_sources),"selected:",rg.tt_sources["sel"].sum())
    
#Subselect sources based on choice
if len(srcs_ids)>0:
    rg.tt_sources.add_index("rg_src_id")
    idx_srcs = rg.tt_sources.loc_indices["rg_src_id", srcs_ids]
    tt_srcs = Table(rg.tt_sources[idx_srcs])
else:
    tt_srcs = rg.tt_sources

nr_srcs = len(tt_srcs)
tt_srcs.add_index("rg_src_id")
#display(tt_srcs)

Nr sources: 10413 selected: 10413


### Show all info for one source

In [3]:
#Define displayed tables and columns
disp_tt_sources = ['rg_src_id','nr_det','ra','dec','pos_err','pos_xv','pos_cpval','pos_rchiq','coadd_src_id','coadd_dist','obs_filter_id','flux','flux_err','flux_nxv','flux_cpval','flux_rchiq','coadd_ffactor',
                   'coadd_fdiff_s2n','nr_fd_srcs','hr','hr_err'] #'fd_src_id','rg_fd_id,,'pos_var','flux_var','sel', ,'simbad_match_id','gaiadr3_match_id'
disp_tt_simbad = ['ogrp','otypes','main_id','otype','ra','dec','ra_prec','dec_prec','coo_err_maja','coo_err_mina','coo_err_angle','coo_qual','coo_wavelength','coo_bibcode','distance_distance','distance_q','distance_unit',
                  'distance_merr','distance_perr','distance_method','distance_bibcode','match_distance','rvz_type','rvz_radvel','rvz_error','rvz_qual','rvz_wavelength','rvz_bibcode','z_value','sp_type','sp_qual',
                  'sp_bibcode','simbad_match_id'] #,'rg_src_id',,'script_number_id'
disp_tt_gaiadr3 = ['ogrp','Gmag_abs','BP-RP','ra','dec','Source','e_RA_ICRS','e_DE_ICRS','Plx','e_Plx','PM','pmRA','e_pmRA','pmDE','e_pmDE','RUWE','FG','e_FG','Gmag','FBP','e_FBP','BPmag','FRP','e_FRP','RPmag','RV','e_RV',
                   'Vbroad','GRVSmag','QSO','Gal','NSS','XPcont','XPsamp','RVS','EpochPh','EpochRV','MCMCGSP','MCMCMSC','And','Teff','logg','__Fe_H_','Dist','A0','HIP','PS1','SDSS13','SKYM2','TYC2','URAT1','AllWISE','APASS9',
                   'GSC23','RAVE5','_2MASS','RAVE6','RAJ2000','DEJ2000','PQSO','PGal','PSS','RPlx','match_distance','gaiadr3_match_id'] #,'_q',,'rg_src_id'
dd_tab_col = {"tt_sources":disp_tt_sources, "tt_simbad":disp_tt_simbad, "tt_gaiadr3":disp_tt_gaiadr3}

def display_src_info(rg_src_id=0):

    #Get source
    tc_src = rg.get_src_from_id(rg_src_id, load_from_file = load_src_from_file, add_sed=show_sed,add_gphoton=False, add_spectrum=show_sed)

    #Prepare figure

    fig, axs = plt.subplots(1,2,figsize=(20, 6),squeeze=True, num="Source ID " + str(rg_src_id), clear=True, gridspec_kw={'width_ratios': [2, 1]})
    axs = axs.flatten()
    
    #Plot light curve und SED
    vvis.plot_light_curve(tc_src, ax = axs[0]) #
    if show_sed:
        vvis.plot_sed(tc_src,ax = axs[1])
    plt.show()

    #If lomb scargle present and signal is at more than 3 sigma plot it too
    if "tt_lombscargle" in tc_src._table_names:
        if tc_src.tt_lombscargle["ls_peak_pval"] < 0.002699796063:
            fig_ls, axs_ls = plt.subplots(1,2,figsize=(15, 5),squeeze=True, num="LS " + str(rg_src_id), clear=True)
            axs_ls = axs_ls.flatten()
            vvis.plot_lombscargle(tc_src.tt_source_lc, ax=axs_ls[0], ax_lc = axs[0], ax_phase = axs_ls[1])
            #vvis.plot_lombscargle(tc_src.tt_source_lc, ax=axs_ls[2],obs_filter="FUV", ax_phase = axs_ls[3])
            plt.show()
            print("\n tt_lombscargle :")
            display(tc_src.tt_lombscargle)

    for table_name, col_names in dd_tab_col.items():
        if table_name in tc_src._table_names:
            print("\n",table_name,":")
            display(tc_src.__dict__[table_name][col_names])


    
    #tc_src.tt_sed.pprint_all()

widgets.interactive(display_src_info, rg_src_id=tt_srcs["rg_src_id"])

interactive(children=(Dropdown(description='rg_src_id', options=(9697, 46900, 48531, 48609, 51884, 57311, 5743…

### Draw light curves

In [4]:
nr_blocks = int(nr_srcs/lc_block_srcs)+1 
def plot_lc_range(block_nr):
    fig_lc = plt.figure("Light curves",figsize=(10, 9), clear=True)
    lc_min = lc_block_srcs * block_nr
    lc_max = lc_min + lc_block_srcs if lc_min + lc_block_srcs<=nr_srcs-1 else nr_srcs
    vvis.plot_light_curves(rg,fig=fig_lc, rg_src_ids = tt_srcs["rg_src_id"][lc_min:lc_max], ylim = [1.2,550])
    plt.show()
    #display(tt_srcs[lc_min:lc_max]) #[show_lc_pars]
widgets.interactive( plot_lc_range, block_nr=(0,nr_blocks-1))

interactive(children=(IntSlider(value=6, description='block_nr', max=12), Output()), _dom_classes=('widget-int…

### Plot skymaps for one source

In [5]:
def plot_sky_src(rg_src_id=0, fd_idx=0, vis_idx = -1):
    
    #Check if src_id exists
    if rg_src_id not in tt_srcs["rg_src_id"]:
        print("Source ID does not exist") 
        return
    
    #Get source and check id field index out of range
    tc_src = rg.get_src_from_id(rg_src_id)
    if fd_idx > len(tc_src.tt_fields)-1:
        print("Field index out of range") 
        return
    
    #Get field to plot and check if visit index out of range
    rg_fd_id = tc_src.tt_fields[fd_idx]["rg_fd_id"]
    fd_src = rg.get_field(rg_fd_id=rg_fd_id, load_method="FITS", add_field=True)
    if vis_idx > len(fd_src.tt_visits)-1:
        print("Visit index out of range") 
        return

    fig_src = plt.figure("Sky map source",figsize=(8, 7), clear=True)

    #Get sky region of source
    sky_center = SkyCoord(tc_src.tt_sources[0]["ra"]*uu.deg, tc_src.tt_sources[0]["dec"]*uu.deg, frame="icrs")
    sky_region_src = CircleSkyRegion(center=sky_center, radius=0.3*uu.arcmin)
     
    #Plot
    plt.cla()
    ax_src, wcs_src = vvis.plot_field_sky_map(fd_src, img_idx=vis_idx, sky_region = sky_region_src, fig=fig_src) #
    
    #If displaying visits, onle show detections in that visit
    sel_det_rg = np.ones(len(rg.tt_detections), dtype=bool)
    sel_det_src = np.ones(len(tc_src.tt_detections), dtype=bool)
    
    # Preselt field detections, can be removed in the future
    #fd_src.tt_detections = fd_src.tt_detections[fd_src.tt_detections["s2n"]>0]
    
    sel_det_fd = np.ones(len(fd_src.tt_detections), dtype=bool)
    if vis_idx > -1:
        vis_id = fd_src.tt_visits["vis_id"][vis_idx]
        sel_det_rg = (rg.tt_detections["vis_id"]==vis_id) * (rg.tt_detections["obs_filter_id"]==dd_filter2id[fd_src.obs_filter])
        sel_det_src = (tc_src.tt_detections["vis_id"]==vis_id) * (tc_src.tt_detections["obs_filter_id"]==dd_filter2id[fd_src.obs_filter])

        fd_det_coords = SkyCoord(fd_src.tt_detections["ra"][sel_det_fd], fd_src.tt_detections["dec"][sel_det_fd], frame="icrs")
        sel_det_fd = ( (fd_src.tt_detections["vis_id"] == vis_id)
                     * (fd_src.tt_detections["obs_filter_id"] == dd_filter2id[fd_src.obs_filter])
                     * sky_region_src.contains(fd_det_coords, wcs_src))
        
    #Show sources
    plt_ref_src_kwargs = {"marker": "x","markersize": 10,"color":"yellow"}
    _ = vvis.plot_sky_sources(rg.tt_sources,tt_det=rg.tt_detections[sel_det_rg], sky_region_wcs = (sky_region_src,wcs_src),only_selected=False) #
    _ = vvis.plot_sky_sources(rg.tt_coadd_sources,tt_det=None, sky_region_wcs = (sky_region_src,wcs_src),only_selected=False, src_id="coadd_src_id", src_kwargs = plt_ref_src_kwargs)
    if vis_idx > -1:
        plot_tds = {"marker":"x","color":"green"}
        _ = vvis.plot_sky_sources(fd_src.tt_detections[sel_det_fd],src_kwargs=plot_tds, sky_region_wcs = (sky_region_src,wcs_src),  src_id="det_id",only_selected=False)
    plt.show()

    # Display sources only for the shown field
    #fd_src_id = rg.tt_src_id_map[sel_fd_src]["fd_src_id"][0]
    if vis_idx > -1:
        print("\nAll detections for this visit in the field:")
        display(fd_src.tt_detections[sel_det_fd])
    
#Decide which sources to show
list_srcs = tt_srcs["rg_src_id"]
if len(srcs_ids)>0:
    list_srcs = srcs_ids

if show_skymaps:
    widget = widgets.interactive( plot_sky_src, rg_src_id=list_srcs,
                        fd_idx=widgets.IntSlider(min=0, max=10, value=0, step=1),
                        vis_idx = widgets.IntSlider(min=-1, max=100, value=-1, step=1)) #visprob_srcids = [1669,1801,2815,3776,4338,6137,] 
    
    controls = widgets.HBox(widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
    output = widget.children[-1]
    display(widgets.VBox([controls, output]))
    # straight NUV, dashed FUV

### Draw sky maps of entire fields

In [6]:
def plot_sky_field(rg_fd_id):
    if rg_fd_id not in rg.tt_fields["rg_fd_id"]:
        print("source ID does not exist") 
        return
    fig_skymap = plt.figure("Sky map",figsize=(9, 8), clear=True) #, constrained_layout=True
    fd = rg.get_field(rg_fd_id=rg_fd_id, load_method="FITS", add_field=True)
    ax, wcs = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = fd.get_sky_region(),fig=fig_skymap) # 
    ax, tt_src_shown = vvis.plot_sky_sources(rg.tt_sources,tt_det=rg.tt_detections, sky_region_wcs = (fd.get_sky_region(),wcs),only_selected=True) # 
    plt.show()
    #display(tt_src_shown) #[show_map_pars]
if show_skymaps: 
    widgets.interactive(plot_sky_field, rg_fd_id=rg.tt_fields["rg_fd_id"])

In [7]:
a = np.array([1,2,3])
b = (a%1.5)/1.5
print(a,b)

[1 2 3] [0.66666667 0.33333333 0.        ]
